# Unit 3 Integrating the CNN Model with Your Drawing App

## Connecting Your Drawing to AI

You've built a web page where users can draw and made the canvas interactive with **JavaScript**. Now, it's time to bring everything together by connecting your drawing canvas to a real **AI model**. In this lesson, you’ll learn how to send your drawing to the server, have a **Convolutional Neural Network (CNN)** analyze it, and display the AI’s guess right on your web page.

### What You'll Learn

In this section, you will:

  * Load a trained **CNN** model in your **Flask** app.
  * Process the drawing sent from the browser so it’s ready for the model.
  * Use the model to predict what the user drew and show the result in the UI.

First, you'll load the trained CNN model in the Flask app like this:

```python
import tensorflow as tf

model = tf.keras.models.load_model('path/to/your/model')
```

This should be done once, typically at the top of your Flask app, so the model is ready to use for predictions.

-----

Next, let's see how the server receives the image from the browser, decodes it, and prepares it for the model:

```python
data_url = request.form.get('image')  # Get the image data sent from the browser (as a data URL)
header, encoded = data_url.split(',', 1)  # Split the data URL to separate the header from the actual image data
img_bytes = base64.b64decode(encoded)  # Decode the base64-encoded image data into bytes
img = Image.open(io.BytesIO(img_bytes))  # Open the image using PIL from the bytes
```

If the image has transparency, you’ll learn how to handle it so the model gets a clean input:

```python
if img.mode in ('RGBA', 'LA') or (img.mode == 'P' and 'transparency' in img.info):
    background = Image.new('RGBA', img.size, (255, 255, 255, 255))  # Create a white background image
    img = Image.alpha_composite(background, img.convert('RGBA'))  # Composite the drawing onto the white background
```

You’ll also see how to convert and resize the image to match the model’s training data:

```python
img = img.convert('L').resize((28, 28))  # Convert to grayscale and resize to 28x28 pixels
```

And how to prepare the image for prediction:

```python
arr = np.array(img, dtype=np.float32) / 255.0  # Convert image to a NumPy array and normalize pixel values to [0, 1]
arr = 1.0 - arr  # Invert colors if needed (so black lines become white, matching training data)
arr = arr.reshape(1, 28, 28, 1)  # Reshape for the model: batch size 1, 28x28 image, 1 channel
```

-----

Finally, you’ll use the model to predict and display the result:

```python
preds = model.predict(arr)  # Get prediction probabilities from the model
idx = np.argmax(preds)  # Find the index of the highest probability (the predicted class)
prediction = categories[idx]  # Get the label for the predicted class
confidence = float(np.max(preds))  # Get the confidence score for the prediction
```

You’ll also learn how to display the AI’s guess and confidence back to the user, making your app feel smart and interactive.

### Why This Is Exciting

This is where your project truly comes to life. By integrating your UI with a real AI model, you’re not just building a drawing app—you’re creating a tool that can “see” and “understand” what users draw. This skill is valuable for many modern applications, from games to educational tools to real-world AI products.

Let’s get started and see your AI in action\!




## Processing Images for AI Recognition

When users draw on the canvas and submit their drawing, the image is sent to the server as a PNG. PNG images can include an alpha channel (transparency), which can cause issues when processing the image for AI recognition. To ensure consistent results, you need to check if the uploaded image has an alpha channel and, if so, composite it over a white background to remove transparency.

Your task: In app/main.py, complete the code that checks if the uploaded image has an alpha channel (such as mode "RGBA" or "LA"). If it does, create a new white background image and paste the original image onto it using the alpha channel as a mask. This will ensure the image is fully opaque and ready for further processing by the AI model.


```python
from flask import Flask, render_template, request
import tensorflow as tf
import numpy as np
from PIL import Image
import base64, io

app = Flask(__name__)
model = tf.keras.models.load_model(
    '/usercode/FILESYSTEM/app/models/drawing_cnn.keras'
)

categories = [
    'house', 'apple', 'car', 'cat',
    'dog', 'flower', 'star', 'tree',
    'bowtie', 'eyeglasses', 'door', 'umbrella'
]

@app.route('/', methods=['GET', 'POST'])
def home():
    prediction = None
    confidence = None

    if request.method == 'POST':
        # Get base64 image data from canvas
        data_url = request.form.get('image')

        # Decode raw image bytes
        header, encoded = data_url.split(',', 1)
        img_bytes = base64.b64decode(encoded)
        img = Image.open(io.BytesIO(img_bytes))

        # TODO: Check if the image has an alpha channel (RGBA or LA) or if it's a palette image with transparency (img.mode is 'P' and 'transparency' in img.info exists)
        if img.mode in ______ or (img.mode == ______ and ______ in img.info):
            # TODO: Create a new white background image using Image.new with mode 'RGBA', original image size, and white color (255, 255, 255, 255)
            background = ______
            img = Image.alpha_composite(background, img.convert('RGBA'))

    return render_template('index.html',
                           prediction=prediction,
                           confidence=confidence)

if __name__ == '__main__':
    app.run(debug=True)

```

```python
from flask import Flask, render_template, request
import tensorflow as tf
import numpy as np
from PIL import Image
import base64, io

app = Flask(__name__)
model = tf.keras.models.load_model(
    '/usercode/FILESYSTEM/app/models/drawing_cnn.keras'
)

categories = [
    'house', 'apple', 'car', 'cat',
    'dog', 'flower', 'star', 'tree',
    'bowtie', 'eyeglasses', 'door', 'umbrella'
]

@app.route('/', methods=['GET', 'POST'])
def home():
    prediction = None
    confidence = None

    if request.method == 'POST':
        # Get base64 image data from canvas
        data_url = request.form.get('image')

        # Decode raw image bytes
        header, encoded = data_url.split(',', 1)
        img_bytes = base64.b64decode(encoded)
        img = Image.open(io.BytesIO(img_bytes))

        # Check if the image has an alpha channel (RGBA or LA) or if it's a palette image with transparency (img.mode is 'P' and 'transparency' in img.info exists)
        if img.mode in ('RGBA', 'LA') or (img.mode == 'P' and 'transparency' in img.info):
            # Create a new white background image using Image.new with mode 'RGBA', original image size, and white color (255, 255, 255, 255)
            background = Image.new('RGBA', img.size, (255, 255, 255, 255))
            img = Image.alpha_composite(background, img.convert('RGBA'))

    return render_template('index.html',
                           prediction=prediction,
                           confidence=confidence)

if __name__ == '__main__':
    app.run(debug=True)

```

## Preparing Images for AI Recognition

## Creating Debug Images for AI Visualization

## Preparing Images for Neural Network Analysis